# Assumptions:

- Gender 2/3 Men 1/3 Women 
- Age Distribution
  - male:
  - ['18-24', '25-34', '35-44', '45-54', '55+']
  - [0.225, 0.545, 0.17, 0.05 , 0.01]
  - female:
  - ['18-24', '25-34', '35-44', '45-54', '55+']
  - [0.355, 0.49, 0.125, 0.025, 0.005]

In [6]:
import random
import numpy as np
import pandas as pd

# Constants
SEED = random.randint(0, 1000000)
NUM_AGENTS = 1000
SIM_DAYS = 31
CYCLES_PER_DAY = 3
TOTAL_CYCLES = SIM_DAYS * CYCLES_PER_DAY

MOTIVES = ['Amusement', 'Ego-Boost', 'Affair', 'Relationship']
GENDERS = ['Male', 'Female']
HAIR_COLORS = ['Blonde', 'Brown', 'Black', 'Red', 'Other']
EYE_COLORS = ['Blue', 'Brown', 'Green', 'Gray', 'Other']


In [7]:
class Agent:
    def __init__(self, agent_id, gender, age, motive):
        self.id = agent_id
        self.gender = gender
        self.age = age 
        self.motive = motive
        self.hair_color = random.choice(HAIR_COLORS)
        self.eye_color = random.choice(EYE_COLORS)
        self.expected_happiness = 0.5
        self.pickiness = np.clip(np.random.normal(0.5, 0.15), 0, 1)
        self.likes_given = 0
        self.matches = set()
        self.messages_sent = 0

    def like_probability(self, other):
        # Simple base probability based on pickiness
        return self.pickiness

    def send_message_probability(self):
        return 0.5 

In [8]:
class Simulation:
    def __init__(self):
        male_age_cats = ['18-24', '25-34', '35-44', '45-54', '55+']
        male_age_probs = [0.225, 0.545, 0.17, 0.05 , 0.01]
        female_age_cats = ['18-24', '25-34', '35-44', '45-54', '55+']
        female_age_probs = [0.355, 0.49, 0.125, 0.025, 0.005]

        self.agents = []
        for i in range(NUM_AGENTS):
            # assign gender with 2/3 chance male
            gender = 'Male' if random.random() < 2/3 else 'Female'
            # sample age category based on gender distribution
            if gender == 'Male':
                age = np.random.choice(male_age_cats, p=male_age_probs)
            else:
                age = np.random.choice(female_age_cats, p=female_age_probs)
            # random motive
            motive = random.choice(MOTIVES)
            self.agents.append(Agent(i, gender, age, motive))
        self.likes = []
        self.matches = []
        self.proposals = 0

    def run(self):
        for _ in range(TOTAL_CYCLES):
            random.shuffle(self.agents)
            for agent in self.agents:

                # Is user Online?
                if random.random() > 0.5:
                    continue

                # Does User have Matches?
                if agent.matches:
                    for match_id in agent.matches:
                        if random.random() < agent.send_message_probability():
                            agent.messages_sent += 1

                # Matching logic
                candidates = [a for a in self.agents
                              if a.gender != agent.gender and a.id != agent.id]
                candidate = random.choice(candidates)
                self.proposals += 1
                if random.random() < agent.like_probability(candidate):
                    agent.likes_given += 1
                    self.likes.append((agent.id, candidate.id))
                    if random.random() < candidate.like_probability(agent):
                        agent.matches.add(candidate.id)
                        candidate.matches.add(agent.id)
                        self.matches.append((agent.id, candidate.id))
                        agent.expected_happiness += 0.02
                        candidate.expected_happiness += 0.02
                    else:
                        agent.expected_happiness -= 0.04
                        
    def metrics(self):
        total_matches = len(self.matches)
        total_likes = len(self.likes)
        # Matches/Like
        match_quote = (total_matches / total_likes * 100)
        # Average expected happiness
        avg_happiness = np.mean([a.expected_happiness for a in self.agents]) * 100
        return match_quote, avg_happiness, total_matches, total_likes

In [9]:
random.seed(SEED)
np.random.seed(SEED)
sim = Simulation()
sim.run()
mq, eh, tm, tl = sim.metrics()
df_results = pd.DataFrame([{
    'Match-Quote (%)': round(mq, 1),
    'Expected Happiness (%)': round(eh, 1),
    'Total Matches': round(tm, 1),
    'Total Likes': round(tl, 1)
}])
print(df_results)

   Match-Quote (%)  Expected Happiness (%)  Total Matches  Total Likes
0             51.0                    51.9          11893        23321
